# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [18]:
using Manifolds
using Manopt
using LinearAlgebra
using NIfTI
using Plots
using LaTeXStrings
using BenchmarkTools

In [19]:
include("../../../src/decompositions/tensors/naive_low_multilinear_rank_approximation.jl")
include("../../../src/decompositions/tensors/curvature_corrected_low_multilinear_rank_approximation.jl")
include("../../../src/decompositions/tensors/iterative_curvature_corrected_low_multilinear_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")

exact_loss (generic function with 2 methods)

### Load data and construct manifold ###

In [20]:
# load data
ni = niread("../../../data/nifti_dt_nonlinear.nii.gz") * 1e9;
size(ni)

(112, 112, 50, 1, 6)

In [21]:
 d1, d2, d3, _, d = size(ni)

# construct data as points on the manifold (and add 1e-5 * I for numerical stability)
predata = [ # data ordered as [xx, yx, yy, zx, zy, zz]
    [
    [ni[i,j,k,1,1] + 1e-5;; ni[i,j,k,1,2];; ni[i,j,k,1,4]]; 
    [ni[i,j,k,1,2];; ni[i,j,k,1,3] + 1e-5;; ni[i,j,k,1,5]]; 
    [ni[i,j,k,1,4];; ni[i,j,k,1,5];; ni[i,j,k,1,6] + 1e-5]
    ] for i=1:d1, j=1:d2, k=1:d3];
    
# pick a 2D slice 
data = predata[38:77,36:75,15];
data_patch = data[1:10,1:10]
println("data size = $(size(data))")
println("data patch size = $(size(data_patch))")

# construct data manifold
n = size(data)
n_patch = size(data_patch)
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M) 

data size = (40, 40)
data patch size = (10, 10)


6

### Construct low rank approximation for full data ###

In [23]:
# Export slice image
asymptote_export_SPD("results/Camino2D_orig.asy", data=data);

In [22]:
unrolled_data = []
for mat in data
    push!(unrolled_data,mat)
end
q = mean(M, unrolled_data)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [25]:
curvature_corrected_low_multilinear_rank_approximation(M, q, data, [5,5])

([[0.8950044738973227 0.6692007023653097 0.18192408878533542; 0.6692007023653097 -0.3959969445677654 -0.1943966809004528; 0.18192408878533542 -0.1943966809004528 0.520395048654941] [4.778954643848296 0.8350167502433133 -0.6096050134116588; 0.8350167502433133 3.4546565459092684 -0.29378223771945977; -0.6096050134116588 -0.29378223771945977 3.591810436594278] … [-1.5789986320626614 1.818457276265771 0.5161712206847788; 1.818457276265771 -1.0787680982280126 -0.6785620671386691; 0.5161712206847788 -0.6785620671386691 -2.029295990487366] [1.2666231856002175 -1.3857754891774232 0.7759527838512013; -1.3857754891774232 0.8360238590419994 -0.5417704520633215; 0.7759527838512013 -0.5417704520633215 -1.1620456970187312]; [-3.4061642776829895 0.6555409784628693 -0.2285094606598303; 0.6555409784628693 -3.4678282342077376 1.0460681109163212; -0.2285094606598303 1.0460681109163212 -4.936097465547379] [2.3640118956035483 1.4383978171194534 -1.0177369726704752; 1.4383978171194534 0.6779202101827806 -0.

In [23]:
# rank_range = collect(1:min(n[1],n[2]))
# rank_range = collect(1:3:20)
rank_range = collect(1:3:min(n[1],n[2]))

ranks = []
nR_q = []
nU = []
ccR_q = []
ccU = []

for rank in rank_range
    push!(ranks,rank)
    println("#$(rank) | computing naive low-rank approximation")
    nRr_q, nUr = naive_low_multilinear_rank_approximation(M, q, data, [rank,rank]); 
    push!(nR_q, nRr_q)
    push!(nU, nUr)
    println("#$(rank) | computing curvature corrected low-rank approximation")
    ccRr_q, ccUr = curvature_corrected_low_multilinear_rank_approximation(M, q, data, [rank,rank]); 
    push!(ccR_q, ccRr_q)
    push!(ccU, ccUr)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
#4 | computing naive low-rank approximation
#4 | computing curvature corrected low-rank approximation
#7 | computing naive low-rank approximation
#7 | computing curvature corrected low-rank approximation
#10 | computing naive low-rank approximation
#10 | computing curvature corrected low-rank approximation
#13 | computing naive low-rank approximation
#13 | computing curvature corrected low-rank approximation
#16 | computing naive low-rank approximation
#16 | computing curvature corrected low-rank approximation
#19 | computing naive low-rank approximation
#19 | computing curvature corrected low-rank approximation
#22 | computing naive low-rank approximation
#22 | computing curvature corrected low-rank approximation
#25 | computing naive low-rank approximation
#25 | computing curvature corrected low-rank approximation
#28 | computing naive low-rank approximation
#28 | computing curvature

LoadError: InterruptException:

In [ ]:
num_ranks = length(ccU)
ref_distance = sum(distance.(Ref(M), Ref(q), data).^2)

naive_tangent_distances_r = zeros(num_ranks)
predicted_naive_distances_r= zeros(num_ranks)
true_naive_distances_r= zeros(num_ranks)

curvature_corrected_tangent_distances_r = zeros(num_ranks)
predicted_curvature_corrected_distances_r = zeros(num_ranks)
true_curvature_corrected_distances_r = zeros(num_ranks)

for (idx,rank) in enumerate(ranks)
    naive_log_q_data_r = Symmetric.([sum([nR_q[idx][i,j] * nU[idx][1][k,i] * nU[idx][2][l,j] for i in 1:rank, j in 1:rank]) for k=1:n[1], l=1:n[2]])
    curvature_corrected_log_q_data_r = Symmetric.([sum([ccR_q[idx][i,j] * ccU[idx][1][k,i] * ccU[idx][2][l,j] for i in 1:rank, j in 1:rank]) for k=1:n[1], l=1:n[2]])
    # expoentiate back
    naive_data_r = exp.(Ref(M), Ref(q), naive_log_q_data_r)
    curvature_corrected_data_r = exp.(Ref(M), Ref(q), curvature_corrected_log_q_data_r)

    # compute relative tangent space error
    naive_tangent_distances_r[idx] = sum(norm.(Ref(M), Ref(q),  log_q_data - naive_log_q_data_r).^2) / ref_distance
    curvature_corrected_tangent_distances_r[idx] = sum(norm.(Ref(M), Ref(q),  log_q_data - curvature_corrected_log_q_data_r).^2) / ref_distance

    # compute relative manifold error
    predicted_naive_distances_r[idx] = curvature_corrected_loss(M, q, data, naive_log_q_data_r)
    true_naive_distances_r[idx] = exact_loss(M, q, data, naive_log_q_data_r)
    predicted_curvature_corrected_distances_r[idx] = curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
    true_curvature_corrected_distances_r[idx] = exact_loss(M, q, data, curvature_corrected_log_q_data_r)
end

In [ ]:
plot(ranks, [naive_tangent_distances_r, true_naive_distances_r, true_curvature_corrected_distances_r], label = ["theoretical lower bound" "naive" "curvature corrected"], ylims=(0,1), xlims=(1,max(ranks...)),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/Camino2D_errors_by_rank.svg")
plot(ranks, [naive_tangent_distances_r .+ 1e-16, true_naive_distances_r .+ 1e-16, true_curvature_corrected_distances_r .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected"], ylims=(1e-16,1), xlims=(1,max(ranks...)), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/Camino2D_logerrors_by_rank.svg")

LoadError: UndefVarError: naive_tangent_distances_r not defined

In [ ]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
# plot(ranks, [predicted_naive_distances_r ./ true_naive_distances_r, predicted_curvature_corrected_distances_r ./ true_curvature_corrected_distances_r], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,max(ranks...)),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
plot(ranks[1:end-1], (predicted_curvature_corrected_distances_r[1:end-1] .- true_curvature_corrected_distances_r[1:end-1] .+ 1e-16) ./ (curvature_corrected_tangent_distances_r[1:end-1] .* sqrt.(curvature_corrected_tangent_distances_r[1:end-1] .* ref_distance) .+ 1e-16), label=("curvature corrected"), xlims=(1,ranks[end-1]),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"), color=3)
savefig("results/Camino2D_discrepancy_by_rank.svg")
# plot(ranks, [predicted_naive_distances_r ./ true_naive_distances_r, predicted_curvature_corrected_distances_r ./ true_curvature_corrected_distances_r], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,max(ranks...)), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
# savefig("results/Camino2D_logdiscrepancy_by_rank.svg")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/2D/P3/results/Camino2D_logdiscrepancy_by_rank.png"

#### Benchmark different methods ####

In [8]:
@benchmark naive_low_multilinear_rank_approximation(M, q, data, [2,2])

BenchmarkTools.Trial: 17 samples with 1 evaluation.
 Range (min … max):  261.216 ms … 333.792 ms  ┊ GC (min … max): 0.00% … 6.07%
 Time  (median):     300.306 ms               ┊ GC (median):    6.10%
 Time  (mean ± σ):   297.645 ms ±  18.973 ms  ┊ GC (mean ± σ):  4.21% ± 3.16%

  ▁▁                 ▁ █  ▁   ▁▁  ▁█ ▁▁  ▁    ▁           ▁   ▁  
  ██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁▁█▁▁▁██▁▁██▁██▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁█ ▁
  261 ms           Histogram: frequency by time          334 ms <

 Memory estimate: 81.78 MiB, allocs estimate: 863128.

In [9]:
@benchmark curvature_corrected_low_multilinear_rank_approximation(M, q, data, [2,2]) 

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  712.979 ms … 758.683 ms  ┊ GC (min … max): 2.56% … 4.97%
 Time  (median):     754.585 ms               ┊ GC (median):    5.07%
 Time  (mean ± σ):   744.111 ms ±  16.563 ms  ┊ GC (mean ± σ):  4.44% ± 1.34%

                                                         █       
  ▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▇ ▁
  713 ms           Histogram: frequency by time          759 ms <

 Memory estimate: 217.18 MiB, allocs estimate: 2290743.

In [15]:
nT = []
nΣ = []

rank_range = collect(1:6:min(n[1],n[2]))

for i in rank_range
    nbm = @benchmark naive_low_multilinear_rank_approximation(M, q, data, [$i, $i])
    push!(nT, mean(nbm).time / 1e9)
    push!(nΣ, std(nbm).time / 1e9)
end

In [12]:
ccT = []
ccΣ = []

for i in rank_range
    ccbm = @benchmark curvature_corrected_low_multilinear_rank_approximation(M, q, data, [$i, $i])
    push!(ccT, mean(ccbm).time / 1e9)
    push!(ccΣ, std(ccbm).time / 1e9)
end

In [17]:
# methods above each other and results per rank in colums
println("tHOSVD" * prod([" & " * raw"$" * "$(Float16(nT[i]))" * raw"$" for i in 1:length(rank_range)]) * raw"\\ ")
println("CC-tHOSVD" * prod([" & " * raw"$" * "$(Float16(ccT[i]))" * raw"$" for i in 1:length(rank_range)]) * raw"\\ ")

tHOSVD & $0.297$ & $0.278$ & $0.2922$ & $0.329$ & $0.4585$ & $0.937$\\ 
CC-tHOSVD & $0.4487$ & $0.763$ & $1.298$ & $1.888$ & $2.896$ & $4.023$\\ 


### Construct low rank approximation for data patch ###

In [31]:
# Export slice image
asymptote_export_SPD("results/Camino2D_patch_orig.asy", data=data_patch);

In [32]:
unrolled_data_patch = []
for mat in data_patch
    push!(unrolled_data_patch,mat)
end
q_patch = mean(M, unrolled_data_patch)
log_q_data_patch = log.(Ref(M), Ref(q_patch), data_patch);  # ∈ T_q P(3)^n

In [33]:
curvature_corrected_low_multilinear_rank_approximation(M, q_patch, data_patch, [5,5])

([[1.456967130844801 1.037992138707543 -0.11840550963060817; 1.0379921387075428 0.16280407999894575 0.23094068297268014; -0.11840550963060817 0.23094068297268014 -0.7272514736731621] [-0.6715330442894946 -0.4844381736383714 0.08096625570505155; -0.4844381736383714 -0.9587158868279703 -0.5800838384837529; 0.08096625570505155 -0.5800838384837529 0.3608374929652587] … [0.8859168288424525 0.6256748595834983 -0.275461134157919; 0.6256748595834982 1.1661001979615557 0.34819272813230634; -0.275461134157919 0.34819272813230634 -0.058904818142000874] [-0.4019241298512537 -0.1401700968336153 0.12370003306713377; -0.1401700968336153 -0.2993226486955483 0.04388092954662832; 0.12370003306713377 0.04388092954662832 0.09271531808771184]; [0.4935451478203173 -0.4229351390287915 -0.28239663734768117; -0.42293513902879143 -0.4561780134204427 -0.2311514790898678; -0.28239663734768117 -0.2311514790898678 0.3747480031689027] [0.7376121080898552 -0.205744245455683 0.4520011334249363; -0.2057442454556829 1.0

In [37]:
rank_range_patch = collect(1:min(n_patch[1],n_patch[2]))

ranks_patch = []
nR_q_patch = []
nU_patch = []
ccR_q_patch = []
ccU_patch = []

for rank in rank_range_patch
    push!(ranks_patch,rank)
    println("#$(rank) | computing naive low-rank approximation")
    nRr_q, nUr = naive_low_multilinear_rank_approximation(M, q_patch, data_patch, [rank,rank]); 
    push!(nR_q_patch, nRr_q)
    push!(nU_patch, nUr)
    println("#$(rank) | computing curvature corrected low-rank approximation")
    ccRr_q, ccUr = curvature_corrected_low_multilinear_rank_approximation(M, q_patch, data_patch, [rank,rank]); 
    push!(ccR_q_patch, ccRr_q)
    push!(ccU_patch, ccUr)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
#2 | computing naive low-rank approximation
#2 | computing curvature corrected low-rank approximation
#3 | computing naive low-rank approximation
#3 | computing curvature corrected low-rank approximation
#4 | computing naive low-rank approximation
#4 | computing curvature corrected low-rank approximation
#5 | computing naive low-rank approximation
#5 | computing curvature corrected low-rank approximation
#6 | computing naive low-rank approximation
#6 | computing curvature corrected low-rank approximation
#7 | computing naive low-rank approximation
#7 | computing curvature corrected low-rank approximation
#8 | computing naive low-rank approximation
#8 | computing curvature corrected low-rank approximation
#9 | computing naive low-rank approximation
#9 | computing curvature corrected low-rank approximation
#10 | computing naive low-rank approximation
#10 | computing curvature corrected l

In [38]:
num_ranks_patch = length(ccU_patch)
ref_distance_patch = sum(distance.(Ref(M), Ref(q_patch), data_patch).^2)

naive_tangent_distances_r_patch = zeros(num_ranks_patch)
predicted_naive_distances_r_patch= zeros(num_ranks_patch)
true_naive_distances_r_patch= zeros(num_ranks_patch)

curvature_corrected_tangent_distances_r_patch = zeros(num_ranks_patch)
predicted_curvature_corrected_distances_r_patch = zeros(num_ranks_patch)
true_curvature_corrected_distances_r_patch = zeros(num_ranks_patch)

for (idx,rank) in enumerate(ranks_patch)
    naive_log_q_data_r_patch = Symmetric.([sum([nR_q_patch[idx][i,j] * nU_patch[idx][1][k,i] * nU_patch[idx][2][l,j] for i in 1:rank, j in 1:rank]) for k=1:n_patch[1], l=1:n_patch[2]])
    curvature_corrected_log_q_data_r_patch = Symmetric.([sum([ccR_q_patch[idx][i,j] * ccU_patch[idx][1][k,i] * ccU_patch[idx][2][l,j] for i in 1:rank, j in 1:rank]) for k=1:n_patch[1], l=1:n_patch[2]])
    # expoentiate back
    naive_data_r_patch = exp.(Ref(M), Ref(q_patch), naive_log_q_data_r_patch)
    curvature_corrected_data_r_patch = exp.(Ref(M), Ref(q_patch), curvature_corrected_log_q_data_r_patch)

    # compute relative tangent space error
    naive_tangent_distances_r_patch[idx] = sum(norm.(Ref(M), Ref(q_patch),  log_q_data_patch - naive_log_q_data_r_patch).^2) / ref_distance_patch
    curvature_corrected_tangent_distances_r_patch[idx] = sum(norm.(Ref(M), Ref(q_patch),  log_q_data_patch - curvature_corrected_log_q_data_r_patch).^2) / ref_distance_patch

    # compute relative manifold error
    predicted_naive_distances_r_patch[idx] = curvature_corrected_loss(M, q_patch, data_patch, naive_log_q_data_r_patch)
    true_naive_distances_r_patch[idx] = exact_loss(M, q_patch, data_patch, naive_log_q_data_r_patch)
    predicted_curvature_corrected_distances_r_patch[idx] = curvature_corrected_loss(M, q_patch, data_patch, curvature_corrected_log_q_data_r_patch)
    true_curvature_corrected_distances_r_patch[idx] = exact_loss(M, q_patch, data_patch, curvature_corrected_log_q_data_r_patch)
end

In [40]:
plot(ranks_patch, [naive_tangent_distances_r_patch, true_naive_distances_r_patch, true_curvature_corrected_distances_r_patch], label = ["theoretical lower bound" "naive" "curvature corrected"], ylims=(0,1), xlims=(1,max(ranks_patch...)),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/Camino2D_patch_errors_by_rank.png")
plot(ranks_patch, [naive_tangent_distances_r_patch .+ 1e-16, true_naive_distances_r_patch .+ 1e-16, true_curvature_corrected_distances_r_patch .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected"], ylims=(1e-16,1), xlims=(1,max(ranks_patch...)), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/Camino2D_patch_logerrors_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/2D/P3/results/Camino2D_patch_logerrors_by_rank.png"

In [43]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
plot(ranks_patch, [predicted_naive_distances_r_patch ./ true_naive_distances_r_patch, predicted_curvature_corrected_distances_r_patch ./ true_curvature_corrected_distances_r_patch], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,max(ranks_patch...)),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/Camino2D_patch_discrepancy_by_rank.png")
plot(ranks_patch, [predicted_naive_distances_r_patch ./ true_naive_distances_r_patch, predicted_curvature_corrected_distances_r_patch ./ true_curvature_corrected_distances_r_patch], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,max(ranks_patch...)), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/Camino2D_patch_logdiscrepancy_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/2D/P3/results/Camino2D_patch_logdiscrepancy_by_rank.png"